<a href="https://colab.research.google.com/github/edchan7-sama/sama-api-colabs/blob/main/Task_Submission_Using_Json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compatible with Google Chrome only.

# Project Settings

In [20]:
from google.colab import files
#@title Step 1 - Set Project Settings { run: "auto", display-mode: "form" }
project_id = "11087" #@param {type:"string"}
client_access_key = "396b087692eee44d84399041800a529d" #@param {type:"string"}
uploaded = files.upload()


Saving 2022-05-07_01-26_34-796_BUG_11087_creation.json to 2022-05-07_01-26_34-796_BUG_11087_creation (3).json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Submit Tasks

In [26]:
filename = "/content/2022-05-07_01-26_34-796_BUG_11087_creation (2).json"
print(f"The following file will be used to grab task ids: ", filename)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Step 2 - Run to submit tasks


In [25]:
import requests 
import json
import time

tasks_moved_to_delivered = []

#filename = next(iter(uploaded))
print(f"The following file will be used to grab task ids: ", filename)

json_dict = json.loads(uploaded[filename].decode("utf-8"))

client_batch_ids = [task_json["data"]["name_scene"] for task_json in json_dict]
print(client_batch_ids)

# get task ids from the user's creation json by matching client_batch_id
task_ids_arr = []
for t_client_batch_id in client_batch_ids:
  try:
    url = f"https://api.sama.com/v2/projects/{project_id}/tasks.json"
   
    print(t_client_batch_id)
    querystring = {
                  "page":1,
                  "page_size":1000,
                  "client_batch_id":t_client_batch_id,
                  "omit_answers":True,
                  "access_key":client_access_key
                   }

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers, params=querystring)

    # Parse `response.text` into JSON
    payload = response.json()
    print(payload)
    if response.status_code == 200:
      if len(payload["tasks"]) != 1 or not payload["tasks"][0]["id"]: 
         raise Exception(f"Error: Invalid json return")
      task_ids_arr.append(payload["tasks"][0]["id"])
    else:
      raise Exception(f"Error: Invalid response code, {response.status_code} : '{response.reason}', '{response.content}")

  except Exception as e:
    print(str(e))
    break

print(task_ids_arr)

for t_task_id in task_ids_arr:
  try:
    # Query URL
    url = f"https://api.sama.com/v2/projects/{project_id}/tasks/{t_task_id}/submit.json?access_key={client_access_key}"

    querystring = { 
      "access_key": client_access_key
    }

    # "submit_latest_answers": True,    
    payload = {
      "submit_latest_answers": True
    }
    
    headers = {
      "Accept": "application/json",
      "Content-Type": "application/json"
    }

    response = requests.put(url, json=payload, headers=headers)

    # Parse `response.text` into JSON
    payload = response.json()

    if response.status_code == 200:
      tasks_moved_to_delivered.append(t_task_id)
    else:
      raise Exception(f"Error: Invalid response code, {response.status_code} : '{response.reason}', '{response.content}")

    # Sleep for a second to not overload API
    time.sleep(1)
  except Exception as e:
    print(str(e))
    break

print(f"Tasks submitted: {len(task_ids_arr)} ")
print(f"Tasks moved to Delivered: {len(tasks_moved_to_delivered)}")



The following file will be used to grab task ids:  2022-05-07_01-26_34-796_BUG_11087_creation (2).json


KeyError: ignored